# Autograd no MXNET
<https://mxnet.apache.org/versions/1.8.0/api/python/docs/tutorials/packages/autograd/index.html>


In [1]:
import mxnet as mx
from mxnet import autograd
from mxnet.gluon.nn import HybridSequential, Dense
from mxnet.gluon.loss import L2Loss
from mxnet import np, npx


## Define Network


### HybridBlcks
<https://gluon.mxnet.io/chapter07_distributed-learning/hybridize.html>

In [2]:
net=HybridSequential()
net.add(Dense(units=3))
net.add(Dense(units=1))
net.initialize()
net.summary

[12:16:30] /home/mertins/Desenvolvimento/C/Terceiros/mxnet/src/storage/storage.cc:205: Using Pooled (Naive) StorageManager for CPU


<bound method Block.summary of HybridSequential(
  (0): Dense(-1 -> 3, linear)
  (1): Dense(-1 -> 1, linear)
)>

### Define função de perda

Função de perda calcula o erro para um único exemplo de treinamento enquanto a função de custo é a média das funções de perda para todo o conjunto de treinamento. 
Isto é verdade?? Acho que muitas vezes são utilizadas de forma similar.

Lost Function == Cost function 


In [3]:
loss_fn=L2Loss()

### Dados ficticios (dummy data) 

In [4]:
# x=mx.np.array([[0.3,0.5]],ctx=npx.gpu(0))
#y=mx.np.array([[1.5]],ctx=npx.gpu(0))
x=mx.np.array([[0.3,0.5]])
y=mx.np.array([[1.5]])

In [5]:
x,y

(array([[0.3, 0.5]]), array([[1.5]]))

In [6]:
with autograd.record():
    y_hat=net(x)
    loss=loss_fn(y_hat,y)
loss.backward()

In [7]:
print(net[0].weight.grad(),net[1].weight.grad(),net[0].bias.data(),net[1].bias.data())

[[-0.0028251  -0.00470849]
 [-0.02185716 -0.0364286 ]
 [ 0.00480542  0.00800903]] [[-0.01281239 -0.04966008 -0.04401871]] [0. 0. 0.] [0.]


In [8]:
net(mx.np.array([[0.3,0.5]]))

array([[0.00135072]])

In [9]:
net[1].bias.data()

array([0.])

## Switching between training vs inference modes

In [11]:
dropout=mx.gluon.nn.Dropout(rate=0.5)
data=mx.np.ones(shape=(3,3))

output=dropout(data)
is_training=autograd.is_training()
print('is_training',is_training,output)


is_training False [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


In [12]:
with autograd.record():
    output=dropout(data)
print('is_training',is_training,output)

is_training False [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 2.]]


In [13]:
dropout.summary

<bound method Block.summary of Dropout(p = 0.5, axes=())>